In [9]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from dlomix.constants import ALPHABET_UNMOD
from prosit_t.layers import (
    MetaEncoder,
    FusionLayer,
    RegressorV2,
    PositionalEmbedding,
    TransformerEncoder,
)

In [10]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [12]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [13]:
SEQ_LENGTH = 30
embeddings_count = len(ALPHABET_UNMOD) + 2
max_ion = SEQ_LENGTH - 1
len_fion = 6

In [79]:
embedding_output_dim = 32
num_heads = 16
ff_dim = 512
transformer_dropout = 0.1
num_transformers = 6
dropout_rate = 0
dense_dim_factor = 4

In [78]:
in_sequence = tf.keras.layers.Input(shape=(30,))
in_collision_energy = tf.keras.layers.Input(shape=(6,))
in_precursor_charge = tf.keras.layers.Input(shape=(1,))

encoded_meta = MetaEncoder(embedding_output_dim*dense_dim_factor, dropout_rate)([in_collision_energy, in_precursor_charge])

x = preprocessing.StringLookup(vocabulary=list(ALPHABET_UNMOD.keys()))(in_sequence)
x = PositionalEmbedding(
            embeddings_count, embedding_output_dim
        )(x)
x = TransformerEncoder(
            embedding_output_dim, num_heads, ff_dim, rate=transformer_dropout, num_transformers=num_transformers
        )(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(embedding_output_dim*dense_dim_factor)(x)
x = tf.keras.layers.Multiply()([x, encoded_meta])
# x = FusionLayer(max_ion)([x, encoded_meta])
x = tf.keras.layers.Flatten()(x)
x = RegressorV2(len_fion * max_ion)(x)

model = tf.keras.Model(inputs=[in_sequence, in_collision_energy, in_precursor_charge],outputs=x)
model.summary()

Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_73 (InputLayer)          [(None, 30)]         0           []                               
                                                                                                  
 string_lookup_24 (StringLookup  (None, 30)          0           ['input_73[0][0]']               
 )                                                                                                
                                                                                                  
 positional_embedding_24 (Posit  (None, 30, 32)      704         ['string_lookup_24[0][0]']       
 ionalEmbedding)                                                                                  
                                                                                           

In [ ]:
## in_sequence = tf.keras.layers.Input(shape=(30,))
# in_collision_energy = tf.keras.layers.Input(shape=(6,))
# in_precursor_charge = tf.keras.layers.Input(shape=(1,))

# encoded_meta = MetaEncoder(embedding_output_dim, dropout_rate)([in_collision_energy, in_precursor_charge])

# x = preprocessing.StringLookup(vocabulary=list(ALPHABET_UNMOD.keys()))(in_sequence)
# x = PositionalEmbedding(
#             embeddings_count, embedding_output_dim
#         )(x)
# x = TransformerEncoder(
#             embedding_output_dim, num_heads, ff_dim, rate=transformer_dropout, num_transformers=num_transformers
#         )(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(embedding_output_dim)(x)
# x = FusionLayer(max_ion)([x, encoded_meta])
# x = TransformerEncoder(
#             embedding_output_dim, num_heads, ff_dim, rate=transformer_dropout, num_transformers=num_transformers
#         )(x)
# x = tf.keras.layers.Flatten()(x)
# x = RegressorV2(len_fion * max_ion)(x)

# model = tf.keras.Model(inputs=[in_sequence, in_collision_energy, in_precursor_charge],outputs=x)
# model.summary()

In [7]:
# in_sequence = tf.keras.layers.Input(shape=(30,))
# in_collision_energy = tf.keras.layers.Input(shape=(6,))
# in_precursor_charge = tf.keras.layers.Input(shape=(1,))

# encoded_meta = MetaEncoder(embedding_output_dim, dropout_rate)([in_collision_energy, in_precursor_charge])

# x = preprocessing.StringLookup(vocabulary=list(ALPHABET_UNMOD.keys()))(in_sequence)
# x = PositionalEmbedding(
#             embeddings_count, embedding_output_dim
#         )(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(embedding_output_dim)(x)
# x = FusionLayer(max_ion)([x, encoded_meta])
# x = TransformerEncoder(
#             embedding_output_dim, num_heads, ff_dim, rate=transformer_dropout, num_transformers=num_transformers
#         )(x)
# x = tf.keras.layers.Flatten()(x)
# x = RegressorV2(len_fion * max_ion)(x)

# model = tf.keras.Model(inputs=[in_sequence, in_collision_energy, in_precursor_charge],outputs=x)
# model.summary()

In [52]:
config = {
    "cyclic_lr": 0,
    "learning_rate": 1e-3
}

In [53]:
optimizer = (
        "adam"
        if "cyclic_lr" in config
        else tf.keras.optimizers.Adam(learning_rate=config["learning_rate"]),
    )[0]

In [54]:
type(optimizer)

str